# RedoxFlow Agent Demonstration

Import the agent class interface

In [13]:
from agent import RedoxFlow

Set REDOXGIT environmental variable to point to the root directory of the repo.

For demonstration, we keep all agent functionality within the repo (e.g., stored memory, resulting filepaths, etc.)

So, we require this environmental variable to be set.

In [14]:
import os
# Local Code Repo Location
os.environ["REDOXGIT"] = "/home/cameron/Documents/Github/RedoxFlow"

In [11]:
# Initialize the agent
rf = RedoxFlow()

Prepare relevant simulation scripts to get all thermodynamic variables needed for redox potential calculation.

This prepare_scripts() method (1) generates reactants, (2) generates reduced product candidates for each reactant, (3) conformer searches both reactants and products, (4) saves the lowest energy reactant/product conformer pair in memory, and then (5) writes the simulation scripts with initial geometries.

In [15]:
agent_raw_output = rf.prepare_scripts(num_generated_reactants=10, # Number of generated reactants to save
                    max_heavy_atoms_per_reactant=16, # Max # of non-hydrogen atoms per reactant
                    min_uniq_elements=1, # Min of non-C elements allowed in the molecule
                    temperature=0.425, # Higher -> More unusual structures, more electronegatives
                    device='auto', # 'cpu', 'cuda:0' for GPUs, 'auto' to auto-determine
                    verbose=False) # Print out intermediate script preparation progress